# Identifier les textes évoquant le concept de république

TODO:
Premier pb ici : vérifier d'ou vient mais
- changement taille fichier entrée (check)
- ex avant de changer cleaning ave le join : 129769 / puis
- VÉRIF ÉCRITURE DU CSV QUI DOIT MERDER ????
- -> C'est le join avant qui visiblement introduit une merde
- mais ne change pas la structure du df avant l'export/reimport
- résolu en forçant le quote à l'export du csv

In [42]:
import pandas as pd

df = pd.read_csv(
    "../data/interim/data_cleaning.csv", low_memory=False, dtype={"ID_orateur": str}
)

In [43]:
# ICI ÇA MERDAIT ENTRE LA SORTIE JUSTE AVANT ET LA LECTURE ET IMPORT DU DF
# Resolu avec quotes
# Voir pourquoi et ou est le pb quand même dans les cols du join > certains noms de sites et réseaux sociaux apparement
df.shape

(129769, 59)

Logique de la tentative :
- regex
- mais exclure certains termes
- mais comme les termes exclus peuvent apparaitre aussi avec les termes voulu, éviter de chainer et finir par virer des trucs qu'on aurait voulu (les idées républicaines sont menacées par Les Républicains)
- Mais comme je suis pas bien sur de moin tentative GPT à vérifier

## INTRODUIRE PRÉ-TRAITEMENT TEXTE

Pour simplifier la vie et faciliter aussi possibles perf d'un futur modèle, virer les parenthèses et balises ici pour s'économiser pas mal de choses du côté des noms de groupes

In [44]:
# nettoyage moche du texte
# sinon à gérer avant !

import re


def nettoyer_texte(texte):
    if not isinstance(texte, str):
        return texte
    # Supprimer les balises HTML/XML
    texte = re.sub(r"<[^>]+>", "", texte)
    # Supprimer contenu entre parenthèses ou crochets
    texte = re.sub(r"\([^)]*\)", "", texte)  # (...)
    # texte = re.sub(r"\[[^\]]*\]", "", texte)     # [...] # TODO: vérif avant
    # Supprimer les espaces multiples
    texte = re.sub(r"\s+", " ", texte).strip()
    return texte


df["Texte_clean"] = df["Texte"].apply(nettoyer_texte)

appliquer le traitement avant fait passer de à 4592 à 4408 (avec la version basique de la regex. Sans doute le groupe socialiste gauche républicaine entre parenthèses, etc.)

In [45]:
# TODO : affiner et notamment les exclusions :
# procureur de la république, etc.

In [46]:
import re

# Regex du champ lexical République (simplifié ici)

# sinon :
pattern_lexical = re.compile(
    # r"(?<!\w)(anti-?|pré-?|post-?|pro?)?(république(s)?|républicain(e)?s?|républicanisme|républicanis(er|ation))\b",
    r"(république(s)?|républicain(e)?s?|républicanisme(s)?|républicanis(er|ation))",
    re.I,
)
# au pire virer le (?<!\w)(anti-?|pré-?|post-?|pro?)?
# genre on raterait irrépublicain
# aussi viré finalement la fin de mot (ce qui permettrait de matcher les s etc. sans se casser les pieds)


# Regex des expressions à exclure
pattern_excl = re.compile(
    r"\b(L|l)es Républicains\b|(P|p)résident de la République"
)  # ici garder la casse


def contains_lexical_outside_excl(text):
    # Trouve toutes les positions des expressions exclues
    excl_positions = [m.span() for m in pattern_excl.finditer(text)]

    # Fonction pour vérifier si une position est dans une zone exclue
    def in_excl(pos):
        for start, end in excl_positions:
            if start <= pos < end:
                return True
        return False

    # Chercher toutes les occurences du champ lexical
    for match in pattern_lexical.finditer(text):
        start_pos = match.start()
        if not in_excl(start_pos):
            return True
    return False


# Appliquer sur la colonne
df["repu_match_valide"] = df["Texte_clean"].apply(contains_lexical_outside_excl)

In [47]:
df[df["repu_match_valide"]].shape

(4409, 61)

In [48]:
# Version bourrin
# Champ lexical : True si le texte contient un terme du champ lexical
lexical_mask = df["Texte_clean"].str.contains(pattern_lexical)

# Exclusion : True si le texte contient une expression à exclure
excl_mask = df["Texte_clean"].str.contains(pattern_excl)

# On garde ceux qui matchent lexicalement mais ne contiennent pas d'exclusion
df["repu_match_simple"] = lexical_mask & ~excl_mask


/var/folders/rq/xsj46x_s2rg87wdksm1_jl3c0000gn/T/ipykernel_4241/1776404343.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  lexical_mask = df["Texte_clean"].str.contains(pattern_lexical)
/var/folders/rq/xsj46x_s2rg87wdksm1_jl3c0000gn/T/ipykernel_4241/1776404343.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  excl_mask = df["Texte_clean"].str.contains(pattern_excl)


In [49]:
df[df["repu_match_simple"]].shape

(3926, 62)

In [50]:
import csv

df_match = df[df["repu_match_valide"]]
df_match.to_csv(
    "../data/interim/df_repu.csv",
    index=False,
    quoting=csv.QUOTE_ALL,  # a permis de résoudre le soucis d'écart. Checker
)


In [51]:
df_match.shape

(4409, 62)

In [52]:
df_test = pd.read_csv("../data/interim/df_repu.csv")

In [53]:
df_test.shape

(4409, 62)